In [9]:
load_trained = True # Carrega o modelo salvo no último treinamento
save_after_each_epoch = True # Salva o modelo a cada epoch concluída
epochs = 1 # Quantidade de epochs do treinamento
colab_enviroment = False # Se estiver rodando no colab, setar como True

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import os
import tensorflow as tf
import itertools
import sklearn.metrics as metrics
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
#Need to edit below import if using base model other than DenseNet
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import Adam, SGD
#Need to edit below import if using base model other than DenseNet
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay 

In [4]:
def make_path(path):
    if colab_enviroment:
        return "/content/drive/MyDrive/FSI/" + path
    else:
        return path

if colab_enviroment:

    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    # unzip dataset
    !unzip -q "/content/drive/MyDrive/FSI/dataset/dataset.zip" -d "/dataset"

In [ ]:
#Setting seeds so that randomization is kept consistent.
np.random.seed(70) 
random.seed(70) 
tf.random.set_seed(70)


#This is the dimensions that each image will be shaped to
#DenseNet201 requires 224 x 224.
img_height, img_width = (224,224)
#Batch size is the number of training examples utilized in one iteration. Could use 16, may increase compute time.
batch_size = 32

#These are the directories/folders where your images are stored for training, validation, and test datasets.
train_data_dir = os.path.join(os.getcwd(), "dataset/train")
valid_data_dir = os.path.join(os.getcwd(), "dataset/test")
test_data_dir = os.path.join(os.getcwd(), "dataset/val")

#ImageDataGenerator performs image augmentation for each image on the fly. Rotating, flipping, brightness, etc.
train_datagen = ImageDataGenerator(
    width_shift_range= 0.2, # 0.2 fraction of total width/height
    height_shift_range=0.2,
    fill_mode="nearest",
    brightness_range=[0.9,1.1], #range for picking a shift value from
    rotation_range =30, #degree range for random rotations
    vertical_flip = True,
    horizontal_flip = True,
    validation_split = 0.05,
    rescale=1./255) #rescaling image pixel values by the number of channels, 1/255

#Pulls your training dataset images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical' #more than 2 classes --> binary
    ) #set as training data

#Pulls your validation dataset images
valid_generator = train_datagen.flow_from_directory(
    valid_data_dir, #same directory as training data
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode= 'categorical') #set as validation data

#Pulls your test dataset images. Note that you only want to use 1 image at a time for test.
test_generator = train_datagen.flow_from_directory(
    test_data_dir, #same directory as training data
    target_size = (img_height, img_width),
    batch_size=1,
    class_mode= 'categorical') #set as validation data


if load_trained:
  # load previous trained model
  model = keras.models.load_model(make_path('saved_models/dense_1118.h5'))
else:
  #Loads array and classes of items in test folder for each batch
  # x,y = test_generator.next()
  # x.shape
  # print("Xshape:", x.shape)
  # print("n_classes: ", valid_generator.num_classes)
  # print("n_classes: ", test_generator.num_classes)
  # print("y: ", y)

  #Sets base model architecture, could swap out DenseNet201 for ResNet50, VGG16, etc
  # Whether to include the fully-connected layer at top of network
  # 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
  base_model = DenseNet201(include_top=False, weights='imagenet')
  #Below x lines are the additional architecture attached beyond the base model
  x = base_model.output
  #Globalaveragepooling performs the 'flatten' purposes
  # x = Flatten()(x)
  x = GlobalAveragePooling2D()(x)
  #128, 64 neuron fully-connected layers with relu activation
  #Dropout for regularization
  x = Dense(128, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(64, activation='relu')(x)
  x = Dropout(0.3)(x)
  #Predicting a label for each image based on softmax activation, for 8 classes
  predictions = Dense(train_generator.num_classes, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)

  #Deciding whether to freeze the base model weights (imagenet) or allow them to update
  #Setting this to 'True' implies we are training the entire model
  for layer in base_model.layers:
      layer.trainable = True

  #Compile the model using stochastic gradient descent w/ learning rate and momentum values
  #Use categorical crossentropy as the loss function
  model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


#CSV logger automatically keeps track of accuracy and loss for both training and validation during each epoch.
#This is very convenient for graphing model performance through time later in R
#Outputs a .csv file
log_csv = CSVLogger('metrics.csv', separator=',', append = False)
callbacks_list = [log_csv]

if save_after_each_epoch:
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=make_path('saved_models/dense_1118.h5'),
      save_weights_only=False,
      verbose=1)

  model_checkpoint_callback_best = tf.keras.callbacks.ModelCheckpoint(
      filepath=make_path('saved_models/best_val_acc.h5'),
      save_weights_only=False,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True,
      initial_value_threshold=0.9,
      verbose=1)
  
  callbacks_list.append(model_checkpoint_callback)
  callbacks_list.append(model_checkpoint_callback_best)

#Train the model for 100 epochs, verbose just tells you much info to display during training
model.fit(train_generator, epochs = epochs, verbose=1, validation_data = valid_generator, callbacks=callbacks_list)

#Save the model and weights
#Mainly need the .h5 file to best hosted on the webapp
model.save_weights(make_path('saved_models/dense_weights_1118'))
model.save(make_path('saved_models/dense_1118.hdf5'))
if not save_after_each_epoch:
  model.save(make_path('saved_models/dense_1118.h5'))